# Managed Lane Example

In [1]:
import os
import geopandas as gpd

from network_wrangler import RoadwayNetwork
from network_wrangler import ProjectCard
from network_wrangler import TransitNetwork
from network_wrangler import Scenario

from lasso import ModelRoadwayNetwork
from lasso import StandardTransit
from lasso import Parameters

%load_ext autoreload
%autoreload 2


### Remote I/O

In [2]:
stpaul_dir = os.path.join(os.getcwd(), '../', 'examples','stpaul')

# Roadway Network files
stpaul_shape_file = os.path.join(stpaul_dir, 'shape.geojson')
stpaul_link_file = os.path.join(stpaul_dir, 'link.json')
stpaul_node_file = os.path.join(stpaul_dir, 'node.geojson')

# Project Card
roadway_project_card_file = os.path.join(
        '../', '../', 'network_wrangler', 'examples', 'stpaul', 'project_cards', '4_simple_managed_lane.yml'
    )

# Test Outputs
test_output_base_file = os.path.join(stpaul_dir, 'test_output_base')
test_output_build_file = os.path.join(stpaul_dir, 'test_output_build')

### Data Reads

In [3]:
roadway_net = RoadwayNetwork.read(
    link_file = stpaul_link_file, 
    node_file = stpaul_node_file, 
    shape_file = stpaul_shape_file, 
    fast = True        
)

transit_net = TransitNetwork.read(feed_path = stpaul_dir)

2020-08-19 15:18:49, INFO: Reading from following files:
-/Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul/link.json
-/Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul/node.geojson
-/Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul/shape.geojson.
2020-08-19 15:18:54, INFO: Read 66253 links from /Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul/link.json
2020-08-19 15:18:54, INFO: Read 17159 nodes from /Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul/node.geojson
2020-08-19 15:18:54, INFO: Read 66253 shapes from /Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul/shape.geojson
2020-08-19 15:19:05, INFO: Read in transit feed from: /Users/wsp/Documents/GitHub/Lasso/notebooks/../examples/stpaul
2020-08-19 15:19:05, INFO: Removing calendar.txt from transit network config because file not found
2020-08-19 15:19:05, INFO: Removing calendar_dates.txt from transit network config because file not found
2020-08-19 15:19

## Apply Project Card to Scenario

In [4]:
base_scenario = {"road_net": roadway_net, "transit_net": transit_net}
working_scenario = Scenario.create_scenario(base_scenario = base_scenario)

2020-08-19 15:19:05, INFO: Creating Scenario


In [5]:
project_card = ProjectCard.read(roadway_project_card_file, validate = False)
working_scenario.apply_project(project_card)

2020-08-19 15:19:05, INFO: Applying test managed lane project
2020-08-19 15:19:05, INFO: Applying Project to Roadway Network: test managed lane project


/Users/wsp/opt/anaconda3/envs/lasso_x/lib/python3.7/site-packages/geopandas/geodataframe.py:853: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super(GeoDataFrame, self).__setitem__(key, value)


In [6]:
working_scenario.applied_projects

['test managed lane project']

## Make Base and Build Travel Model Networks

In [7]:
base_m_net = ModelRoadwayNetwork.from_RoadwayNetwork(roadway_network_object = roadway_net)
build_m_net = ModelRoadwayNetwork.from_RoadwayNetwork(roadway_network_object = working_scenario.road_net)

2020-08-19 15:19:15, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso
2020-08-19 15:19:25, INFO: Lasso base directory set as: /Users/wsp/Documents/GitHub/Lasso


In [8]:
base_m_net.roadway_standard_to_met_council_network()
build_m_net.roadway_standard_to_met_council_network()

2020-08-19 15:19:25, INFO: Renaming roadway attributes to be consistent with what metcouncil's model is expecting
2020-08-19 15:19:25, INFO: Didn't detect managed lanes in network.
2020-08-19 15:19:25, INFO: Creating calculated roadway variables.
2020-08-19 15:19:25, INFO: Calculating Area Type from Spatial Data and adding as roadway network variable: area_type


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:401: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


2020-08-19 15:19:32, INFO: Finished Calculating Area Type from Spatial Data into variable: area_type
2020-08-19 15:19:32, INFO: Adding roadway network variable for county using a spatial join with: /Users/wsp/Documents/GitHub/Lasso/metcouncil_data/county/cb_2017_us_county_5m.shp


/Users/wsp/Documents/GitHub/Lasso/lasso/roadway.py:293: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  centroids_gdf["geometry"] = centroids_gdf["geometry"].centroid


ValueError: cannot reindex from a duplicate axis

### Write to GeoJSON and Shapefile

In [ ]:
out_cols = ['model_link_id', 'shape_id', 'segment_id', 'distance', 'lanes_AM', 'lanes_MD', 'ML_lanes_AM', 'ML_lanes_MD', 'geometry']

base_output = base_m_net.links_df[out_cols][(base_m_net.links_df['drive_access'] == 1) & (base_m_net.links_df['centroidconnect'] == 0)]
base_output.to_file(test_output_base_file + '.geojson', driver = 'GeoJSON')

build_output = build_m_net.links_df[out_cols][(build_m_net.links_df['drive_access'] == 1) & (build_m_net.links_df['centroidconnect'] == 0)]
build_output.to_file(test_output_build_file + '.geojson', driver = 'GeoJSON')

In [ ]:
base_m_net.write_roadway_as_shp(
    output_link_shp = (test_output_base_file + '_links.shp'),
    output_node_shp = (test_output_base_file + '_nodes.shp'),
    link_output_variables = out_cols,
    data_to_dbf = True,
    data_to_csv = False,
)

build_m_net.write_roadway_as_shp(
    output_link_shp = (test_output_build_file + '_links.shp'),
    output_node_shp = (test_output_build_file + '_nodes.shp'),
    link_output_variables = out_cols,
    data_to_dbf = True,
    data_to_csv = False,
)